In [698]:
#IMPORT MODULES
import numpy as np
import pandas as pd
import random
import math
import copy
import re
import sys
import sklearn
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data

**Import Data

In [699]:
gameresults = pd.read_csv("data/gameresults.csv", encoding="latin1").values
gamescores = pd.read_csv("data/full_sheet_scaled.csv", encoding="latin1").fillna(0).values
#gamescores = pd.read_csv("~/Downloads/full_sheet_scaled.csv", encoding="latin1").fillna(0).values[]
mpval = pd.read_csv("data/mp538.csv", encoding="latin1").values
cdf = pd.read_csv("data/cdf.csv",encoding="latin1").values
mindf = pd.read_csv("data/minutes_target.csv",encoding="latin1").values

with open("data/nba_draft.p","rb") as f:
    draft = pickle.load(f)

with open("data/nba_height_weight.p","rb") as f:
    heightweight = pickle.load(f)

with open("data/nba_team_to_abbr_mapping.p","rb") as f:
    team_to_abbr_mapping = pickle.load(f)

In [700]:
MIN_MINUTES = 5
MINUTES_IDX = 47
GAME_ID_IDX = 40
TEAM_IDX = 37
PLAYER_IDX = 0
TOTAL_TEAM_MINUTES = 240

keep = [2,3,5,6,8,9,11,12,14,15,16,17,18,33,35,42,43,44] # box score lines we're including

injured_players = {
    "Klay Thompson",
    "Jonathan Isaac",
    "Markelle Fultz",
    "Spencer Dinwiddie",
    "Thomas Bryant",
    "Marquese Chriss",
    "Chris Clemons"
}

players_to_correct = {
    "Michael Porter": "Jr.",
    "Kevin Porter": "Jr.",
    "Otto Porter":  "Jr.",
    "Tim Hardaway": "Jr.",
    "Gary Trent": "Jr.",
    "Marvin Bagley": "III",
    "Jaren Jackson": "Jr.",
    "Kelly Oubre": "Jr.",
}

East=['BRK','PHI','CHI','NYK','ATL','DET','CLE','ORL','TOR','MIA','CHO','WAS','BOS','IND','MIL']
West=['LAL','UTA','MEM','NOP','GSW','HOU','DAL','DEN','SAC','OKC','POR','MIN','PHO','LAC','SAS']
NBA = East + West

**RUN THESE ONLY TO RETRAIN

In [49]:
box_score_datasets = create_box_score_datasets()

In [50]:
box_score_model = create_and_train_box_score_model(datasets=box_score_datasets, use_all=True, num_epochs=10) #14

0.5705108179108185
0.5411429907891805
0.5391753803343833
0.5384655250079867
0.5380126576980248
0.5377605655974075
0.5375809264173653
0.537383306388031
0.5372851970957733
0.5371627622839659


In [701]:
##RUN THIS DAILY
embeddings, gp = create_embeddings_and_games_played(box_score_model)


149320.46360333517


In [52]:
mp = create_minutes_played(embeddings)

In [53]:
affiliations = create_affiliations(embeddings)

40 Paul Millsap
40 Deron Williams
40 Wesley Matthews
40 C.J. Miles
40 Mehmet Okur
40 Kosta Koufos
40 Kyle Korver
40 Othyus Jeffers
40 Kyrylo Fesenko
40 Ronnie Price
40 Sundiata Gaines
40 Amar'e Stoudemire
40 Steve Nash
40 Jason Richardson
40 Grant Hill
40 Jarron Collins
40 Channing Frye
40 Jared Dudley
40 Leandro Barbosa
40 Goran DragiÄ
40 Lou Amundson
40 Earl Clark
40 Dwayne Jones
40 Joe Johnson
40 Josh Smith
40 Al Horford
40 Mike Bibby
40 Marvin Williams
40 Jamal Crawford
40 Zaza Pachulia
40 Maurice Evans
40 Jeff Teague
40 Joe Smith
40 Mario West
40 John Salmons
40 Brandon Jennings
40 Luc Mbah a Moute
40 Kurt Thomas
40 Carlos Delfino
40 Jerry Stackhouse
40 Ersan Ä°lyasova
40 Luke Ridnour
40 Dan Gadzuric
40 Rajon Rondo
40 Paul Pierce
40 Kevin Garnett
40 Ray Allen
40 Kendrick Perkins
40 Tony Allen
40 Glen Davis
40 Rasheed Wallace
40 Michael Finley
40 Nate Robinson
40 Dwyane Wade
40 Carlos Arroyo
40 Jermaine O'Neal
40 Michael Beasley
40 Quentin Richardson
40 Dorell Wright
40 Udonis Has

40 Daniel Gibson
40 Tony Parker
40 Manu GinÃ³bili
40 Tim Duncan
40 Antonio McDyess
40 Richard Jefferson
40 George Hill
40 Matt Bonner
40 DeJuan Blair
40 Garrett Temple
40 Keith Bogans
40 Grant Hill
40 Steve Nash
40 Amar'e Stoudemire
40 Jason Richardson
40 Jarron Collins
40 Channing Frye
40 Jared Dudley
40 Leandro Barbosa
40 Goran DragiÄ
40 Lou Amundson
40 Al Horford
40 Josh Smith
40 Joe Johnson
40 Marvin Williams
40 Mike Bibby
40 Jamal Crawford
40 Zaza Pachulia
40 Jeff Teague
40 Maurice Evans
40 Randolph Morris
40 Dwight Howard
40 Rashard Lewis
40 Jameer Nelson
40 Matt Barnes
40 Vince Carter
40 J.J. Redick
40 MickaÃ«l PiÃ©trus
40 Jason Williams
40 Ryan Anderson
40 Marcin Gortat
40 Carlos Boozer
40 Wesley Matthews
40 Deron Williams
40 C.J. Miles
40 Kyrylo Fesenko
40 Paul Millsap
40 Kyle Korver
40 Andrei Kirilenko
40 Ronnie Price
40 Kobe Bryant
40 Pau Gasol
40 Metta World Peace
40 Derek Fisher
40 Andrew Bynum
40 Lamar Odom
40 Shannon Brown
40 Jordan Farmar
40 Josh Powell
40 Rajon Rondo


40 Paul Pierce
40 Ray Allen
40 Rajon Rondo
40 Kevin Garnett
40 Kendrick Perkins
40 Rasheed Wallace
40 Tony Allen
40 Glen Davis
40 Nate Robinson
40 Kobe Bryant
40 Pau Gasol
40 Metta World Peace
40 Derek Fisher
40 Andrew Bynum
40 Lamar Odom
40 Jordan Farmar
40 Sasha VujaÄiÄ
40 Luke Walton
40 Shannon Brown
40 Pau Gasol
40 Kobe Bryant
40 Metta World Peace
40 Andrew Bynum
40 Derek Fisher
40 Lamar Odom
40 Shannon Brown
40 Jordan Farmar
40 Sasha VujaÄiÄ
40 Josh Powell
40 Luke Walton
40 Didier Ilunga-Mbenga
40 Ray Allen
40 Paul Pierce
40 Rajon Rondo
40 Kevin Garnett
40 Kendrick Perkins
40 Glen Davis
40 Rasheed Wallace
40 Tony Allen
40 Nate Robinson
40 Shelden Williams
40 Marquis Daniels
40 Michael Finley
40 Metta World Peace
40 Kobe Bryant
40 Pau Gasol
40 Derek Fisher
40 Andrew Bynum
40 Lamar Odom
40 Jordan Farmar
40 Shannon Brown
40 Sasha VujaÄiÄ
40 Josh Powell
40 Paul Pierce
40 Ray Allen
40 Rajon Rondo
40 Kevin Garnett
40 Rasheed Wallace
40 Glen Davis
40 Tony Allen
40 Nate Robinson
40 

In [54]:
adjustment_datasets = create_year_adjustment_datasets(embeddings)

In [55]:
model_players = create_and_train_year_adjustment_model(adjustment_datasets, use_all=True, num_epochs=150, batch_size=64)

673.6738228070878
618.617868282212
568.0574273849189
521.7375124705294
480.20641007380146
441.9724885486982
406.8789696563842
374.5903715601286
345.5008485337724
317.9792697629041
293.53475257085864
270.71647231592016
250.07279106526966
230.78300236396922
213.06494021283
196.62802419131486
181.4890592221541
167.65055978076637
154.96899849826696
143.23100290850383
132.20005077720143
122.17857567342448
112.68104480844241
104.16041684787032
96.44149976997527
89.04650053615659
82.13083464712048
75.76914512959675
69.90020307666128
64.44171716699701
59.414206072777816
54.703433613628775
50.42158338191399
46.40428093756097
42.60212386125542
39.249312969782444
36.01778079351999
33.12573483246761
30.358120537337687
27.854401564461693
25.47601958230124
23.402858986310633
21.39844603986083
19.54445051004958
17.812233295746367
16.302649682598
14.88035067852715
13.545900533258932
12.336554538229729
11.255583770008016
10.215841223197508
9.330663039303616
8.478003550326472
7.725210360658997
7.0090801

In [56]:
embeddings = update_embeddings_with_predictions(model_players, embeddings, coeff=1.0)

In [57]:
team_datasets = create_team_data_sets(gameresults, affiliations)

In [58]:
pc_model = create_and_train_team_model(team_datasets, use_all=True, num_epochs=14, just_minutes=False)

0.24249875971834559 0.5084495069324106
0.23050525527283472 0.5270402637368815
0.22206926357047496 0.5429949368831616
0.2117239603146876 0.5562347913709815
0.2035318557921282 0.5735759403970506
0.19944422623192204 0.5862602749884417
0.19740301827088072 0.5944971188830869
0.19631699218263488 0.5997863682571817
0.19568234730952314 0.6031895878810237
0.19527540624683268 0.6054032997808595
0.19499441468193074 0.606875783579361
0.19478972950360038 0.607888198417166
0.19463476773949792 0.6086120043975719
0.19451394791076052 0.6091499057944846


In [449]:
heightweight, mp = add_inseason_trades()
team_to_player_map, player_to_team_map = create_team_to_player_map(embeddings, heightweight, mp)

checkmark
check2
done


In [707]:
team_abilities = create_team_abilities(pc_model, heightweight, embeddings, mp, team_to_player_map, '2020')

In [708]:
team_win_prob_matrix = create_team_win_probability_matrix(pc_model, team_abilities)

In [709]:
champions, finals, playoffs = simulate_seasons(team_win_prob_matrix, num_simulations=10_000, include_season_to_date=True)

In [710]:
expected_wins = find_expected_wins(gameresults, include_season_to_date=True, display=True)

1 LAL 56.52
2 LAC 53.91
3 BRK 50.79
4 POR 48.3
5 UTA 48.21
6 MIL 45.5
7 PHO 44.86
8 DAL 43.45
9 IND 43.43
10 TOR 42.03
11 GSW 39.64
12 PHI 39.48
13 SAC 39.26
14 SAS 39.11
15 ATL 38.29
16 MEM 37.82
17 DEN 37.59
18 BOS 37.49
19 CHI 36.24
20 HOU 35.91
21 MIA 34.63
22 NOP 33.16
23 CLE 32.86
24 CHO 31.91
25 ORL 30.3
26 OKC 26.27
27 NYK 25.62
28 MIN 25.12
29 WAS 24.39
30 DET 15.9


In [702]:
best_current_players = find_best_current_players('current', box_score_model, display=True)

1 James Harden 0.391
2 Damian Lillard 0.387
3 LeBron James 0.384
4 Nikola JokiÄ 0.374
5 Luka DonÄiÄ 0.364
6 Kawhi Leonard 0.36
7 Stephen Curry 0.351
8 Kevin Durant 0.345
9 Nikola VuÄeviÄ 0.344
10 Kyle Lowry 0.335
11 Karl-Anthony Towns 0.33
12 Anthony Davis 0.322
13 Jayson Tatum 0.321
14 Kyrie Irving 0.317
15 Chris Paul 0.311
16 Giannis Antetokounmpo 0.306
17 Ben Simmons 0.305
18 Khris Middleton 0.294
19 Fred VanVleet 0.293
20 Domantas Sabonis 0.292
21 Paul George 0.29
22 Kristaps PorziÅÄ£is 0.29
23 Rudy Gobert 0.289
24 CJ McCollum 0.289
25 Kevin Love 0.284
26 Jimmy Butler 0.283
27 Clint Capela 0.275
28 Bam Adebayo 0.275
29 Jrue Holiday 0.271
30 Malcolm Brogdon 0.27
31 Shai Gilgeous-Alexander 0.27
32 Joel Embiid 0.252
33 Christian Wood 0.251
34 Zach LaVine 0.248
35 Trae Young 0.247
36 Gordon Hayward 0.246
37 Jarrett Allen 0.246
38 John Collins 0.244
39 Larry Nance 0.243
40 DeMar DeRozan 0.242
41 Bradley Beal 0.239
42 Julius Randle 0.239
43 Jaylen Brown 0.239
44 Dejounte Murray 0.2

488 Sebastian Telfair -0.106
489 Evan Turner -0.106
490 Norris Cole -0.106
491 Iman Shumpert -0.106
492 Troy Williams -0.106
493 Isaiah Taylor -0.106
494 Abdel Nader -0.106
495 Keita Bates-Diop -0.106
496 Roy Hibbert -0.107
497 Thomas Robinson -0.107
498 Joel Freeland -0.107
499 Rodney McGruder -0.107
500 Troy Murphy -0.108
501 Raja Bell -0.108
502 Phil Pressey -0.108
503 Kris Humphries -0.109
504 Rasual Butler -0.109
505 Luther Head -0.109
506 Thabo Sefolosha -0.109
507 Joey Dorsey -0.109
508 Anthony Randolph -0.11
509 Damien Wilkins -0.11
510 Willie Reed -0.11
511 Luc Mbah a Moute -0.111
512 Greg Monroe -0.111
513 Randy Foye -0.112
514 Derek Fisher -0.112
515 James Posey -0.112
516 Dante Exum -0.112
517 DeAndre' Bembry -0.112
518 JosÃ© CalderÃ³n -0.113
519 Rashard Lewis -0.113
520 Shelvin Mack -0.113
521 Ray McCallum -0.113
522 Markel Brown -0.113
523 Jahlil Okafor -0.113
524 Emmanuel Mudiay -0.113
525 MiloÅ¡ TeodosiÄ -0.113
526 Boris Diaw -0.114
527 Joel Anthony -0.114
528 C.J. Mil

In [790]:
most_improved_player()

1 LaMelo Ball 4.8
2 Chris Boucher 4.1
3 Julius Randle 3.5
4 Jaylen Brown 3.5
5 De'Andre Hunter 3.4
6 Dejounte Murray 3.1
7 Malcolm Brogdon 2.9
8 CJ McCollum 2.9
9 Donovan Mitchell 2.7
10 Nikola JokiÄ 2.7
11 Luguentz Dort 2.7
12 RJ Barrett 2.6
13 Tyrese Haliburton 2.6
14 Harrison Barnes 2.5
15 Jerami Grant 2.4
16 Myles Turner 2.4
17 Joel Embiid 2.3
18 Mike Conley 2.3
19 Mikal Bridges 2.3
20 Jordan Clarkson 2.2
21 Jayson Tatum 2.1
22 Shai Gilgeous-Alexander 2.1
23 Deandre Ayton 1.9
24 Zach LaVine 1.8
25 OG Anunoby 1.8
26 Bam Adebayo 1.8
27 Bobby Portis 1.8
28 Kentavious Caldwell-Pope 1.8
29 Christian Wood 1.8
30 Victor Oladipo 1.8
31 Nicolas Batum 1.8
32 Kawhi Leonard 1.8
33 Nikola VuÄeviÄ 1.7
34 Anthony Davis 1.7
35 Richaun Holmes 1.7
36 Joe Harris 1.6
37 Kyle Anderson 1.6
38 Zion Williamson 1.6
39 P.J. Washington 1.5
40 Eric Gordon 1.5
41 Kyrie Irving 1.4
42 Tyler Herro 1.4
43 Kevin Durant 1.3
44 Coby White 1.3
45 Blake Griffin 1.3
46 Darius Garland 1.3
47 Deni Avdija 1.3
48 Malik B

[('LaMelo Ball', 4.8),
 ('Chris Boucher', 4.1),
 ('Julius Randle', 3.5),
 ('Jaylen Brown', 3.5),
 ("De'Andre Hunter", 3.4),
 ('Dejounte Murray', 3.1),
 ('Malcolm Brogdon', 2.9),
 ('CJ McCollum', 2.9),
 ('Donovan Mitchell', 2.7),
 ('Nikola JokiÄ\x87', 2.7),
 ('Luguentz Dort', 2.7),
 ('RJ Barrett', 2.6),
 ('Tyrese Haliburton', 2.6),
 ('Harrison Barnes', 2.5),
 ('Jerami Grant', 2.4),
 ('Myles Turner', 2.4),
 ('Joel Embiid', 2.3),
 ('Mike Conley', 2.3),
 ('Mikal Bridges', 2.3),
 ('Jordan Clarkson', 2.2),
 ('Jayson Tatum', 2.1),
 ('Shai Gilgeous-Alexander', 2.1),
 ('Deandre Ayton', 1.9),
 ('Zach LaVine', 1.8),
 ('OG Anunoby', 1.8),
 ('Bam Adebayo', 1.8),
 ('Bobby Portis', 1.8),
 ('Kentavious Caldwell-Pope', 1.8),
 ('Christian Wood', 1.8),
 ('Victor Oladipo', 1.8),
 ('Nicolas Batum', 1.8),
 ('Kawhi Leonard', 1.8),
 ('Nikola VuÄ\x8deviÄ\x87', 1.7),
 ('Anthony Davis', 1.7),
 ('Richaun Holmes', 1.7),
 ('Joe Harris', 1.6),
 ('Kyle Anderson', 1.6),
 ('Zion Williamson', 1.6),
 ('P.J. Washington', 

In [711]:
sorted_results = sort_results(champions)

1 BRK 0.6142
2 LAL 0.0921
3 POR 0.0803
4 LAC 0.0733
5 TOR 0.0409
6 IND 0.0233
7 PHO 0.0228
8 UTA 0.017
9 MIL 0.0137
10 DAL 0.0113
11 SAC 0.0079
12 MIA 0.0009
13 ATL 0.0008
14 GSW 0.0004
15 HOU 0.0004
16 CHI 0.0002
17 BOS 0.0002
18 PHI 0.0001
19 NOP 0.0001
20 SAS 0.0001
21 NYK 0.0
22 DET 0.0
23 CLE 0.0
24 ORL 0.0
25 CHO 0.0
26 WAS 0.0
27 MEM 0.0
28 DEN 0.0
29 OKC 0.0
30 MIN 0.0


In [712]:
finals_results = sort_results(finals)

1 BRK 0.7771
2 LAL 0.2696
3 POR 0.2379
4 LAC 0.225
5 TOR 0.0967
6 PHO 0.0939
7 UTA 0.0723
8 IND 0.0631
9 DAL 0.0532
10 MIL 0.0453
11 SAC 0.036
12 ATL 0.0062
13 GSW 0.0054
14 MIA 0.0045
15 CHI 0.0042
16 HOU 0.0038
17 BOS 0.0017
18 PHI 0.001
19 MEM 0.0009
20 NOP 0.0009
21 SAS 0.0008
22 DEN 0.0003
23 CLE 0.0001
24 CHO 0.0001
25 NYK 0.0
26 DET 0.0
27 ORL 0.0
28 WAS 0.0
29 OKC 0.0
30 MIN 0.0


In [713]:
playoff_results = sort_results(playoffs)

1 BRK 1.0
2 LAL 0.9991
3 LAC 0.9989
4 POR 0.9941
5 TOR 0.9932
6 MIL 0.9932
7 IND 0.9926
8 UTA 0.9716
9 PHO 0.942
10 DAL 0.8482
11 ATL 0.7926
12 SAC 0.6982
13 PHI 0.6881
14 CHI 0.6839
15 MIA 0.6557
16 BOS 0.6273
17 GSW 0.5234
18 CLE 0.2941
19 HOU 0.2933
20 SAS 0.2757
21 MEM 0.205
22 CHO 0.1463
23 NOP 0.1366
24 ORL 0.1178
25 DEN 0.1114
26 WAS 0.0144
27 MIN 0.002
28 NYK 0.0008
29 OKC 0.0005
30 DET 0.0


In [642]:
#history of players
for player in embeddings:
    player_history(player)

**START OF UTILITY CODE

In [4]:
def run_through_model(model, dataset, batch_size, num_epochs, optimizer=None, regularize=False):
    criterion = nn.MSELoss()
    sampler = data.RandomSampler(dataset)
    data_loader = data.DataLoader(dataset=dataset, batch_size=batch_size, sampler=sampler, drop_last=True)
    for epoch in range(num_epochs):
        total_loss = 0
        for (batch, labels) in data_loader:
            prediction = model(batch)
            regularization_penalty = (0.25 * torch.norm(prediction)) if regularize else torch.zeros(1).detach()
            loss = criterion(prediction, labels) + regularization_penalty
            total_loss += loss.item() - regularization_penalty.item()
            if optimizer != None:
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
        print(total_loss / (int(len(dataset) / batch_size)))

**MODEL FOR ENCODING BOX SCORES AS EMBEDDINGS (DECLARATION FOLLOWED BY TRAINING)

In [5]:
##Train Embeddings
class BoxScoreToEmbeddingModel(nn.Module):
    
    def __init__(self, dimensionality):
        super().__init__()
        self.__layer_one = nn.Linear(dimensionality, 4)
        self.__layer_two = nn.Linear(4, 1)
    
    def forward(self, performance):
        if (len(performance) == 1):
            performance.unsqueeze(0)
        return self.predict(self.encode(performance))
    
    def encode(self,performance):
        return self.__layer_one(performance)
    
    def predict(self, embedding):
        return self.__layer_two(torch.tanh(embedding))

In [6]:
class PlayerBoxScoreDataSet(data.Dataset):
        
    def __init__(self, box_score_list, min_minutes=10, max_minutes=40):
        super().__init__()        
        LABEL_IDX = 46
        self.__x = self.__filter_games(box_score_list, keep, min_minutes, max_minutes)
        self.__y = self.__filter_games(box_score_list, [LABEL_IDX], min_minutes, max_minutes)
        
    def __filter_games(self, games, indices, min_minutes, max_minutes):
        return torch.stack([torch.tensor(game[indices].astype("double"), dtype=torch.double) for game in filter(lambda x: x[MINUTES_IDX] > min_minutes and x[MINUTES_IDX] < max_minutes, games)])
    
    def __getitem__(self, idx):
        return (self.__x[idx], self.__y[idx])
    
    def __len__(self):
        return len(self.__x)

In [7]:
def create_box_score_datasets():
    TEST_TRAIN_DIVIDE_IDX = int(0.8 * len(gamescores))
    return {
        "overall": PlayerBoxScoreDataSet(gamescores),
        "training": PlayerBoxScoreDataSet(gamescores[0:TEST_TRAIN_DIVIDE_IDX]),
        "validation": PlayerBoxScoreDataSet(gamescores[TEST_TRAIN_DIVIDE_IDX:len(gamescores)])
    }

In [8]:
def train_box_score_model(box_score_model, datasets, use_all=False, num_epochs=10, batch_size=64, regularize=False):
    box_score_optimizer = optim.AdamW(box_score_model.parameters(), lr=1e-4)
    with torch.enable_grad():
        run_through_model(box_score_model, datasets["training"] if not use_all else datasets["overall"], batch_size=batch_size, num_epochs=num_epochs, optimizer=box_score_optimizer, regularize=regularize)

def validate_box_score_model(box_score_model, datasets):
    with torch.no_grad():
        run_through_model(box_score_model, datasets["validation"], batch_size=1, num_epochs=1)

def create_and_train_box_score_model(datasets, use_all=False, num_epochs=10, batch_size=64, regularize=False):
    box_score_model = BoxScoreToEmbeddingModel(18).double() 
    train_box_score_model(box_score_model, datasets, use_all=use_all, num_epochs=num_epochs, batch_size=batch_size, regularize=regularize)
    if not use_all:
        validate_box_score_model(box_score_model, datasets)
    return box_score_model

**CALCULATES PLAYER'S EMBEDDINGS

In [9]:
def get_player_name(player):
    return player + " " + players_to_correct[player] if player in players_to_correct else player

def correct_player_dict(player_dict):
    for player in players_to_correct:
        player_dict[player + " " + players_to_correct[player]] = embeddings[player]

In [10]:
def create_embeddings_and_games_played(box_score_model):
    
    year_endings_map = {
        gameresults[1170, 0]: '2010',
        gameresults[2543, 0]: '2011',
        gameresults[3559, 0]: '2012',
        gameresults[4926, 0]: '2013',
        gameresults[6241, 0]: '2014',
        gameresults[7559, 0]: '2015',
        gameresults[8871, 0]: '2016',
        gameresults[10187, 0]: '2017',
        gameresults[11496, 0]: '2018',
        gameresults[12808, 0]: '2019',
        gameresults[13949, 0]: '2020'
    }

    embeddings={}
    gp={}
    loss_embeddings = 0
    counter = 0

    BASE_UPDATE_COEFFICIENT = (1/4) #TRYING OUT DIFFERENT CO-EFFICIENTS

    for game in gamescores:
        player = get_player_name(game[PLAYER_IDX])
        embeddings[player]={'current': torch.zeros(4)}
        gp[player] = 0

    old_game_id = None
    for game in filter(lambda x: x[MINUTES_IDX] > MIN_MINUTES, gamescores):

        game_id = game[GAME_ID_IDX]
        if game_id != old_game_id and game_id in year_endings_map:
            for player_name in embeddings:
                embeddings[player_name][year_endings_map[game_id]] = copy.deepcopy(embeddings[player_name]['current'])

        player = get_player_name(game[PLAYER_IDX])
        game_val_input = torch.from_numpy(game[keep].astype('double')).detach()
        with torch.no_grad():               
            encoded_game_val= box_score_model.encode(game_val_input)

        prev_gp = gp[player]
        new_gp = prev_gp + 1
        gp[player] = new_gp

        update_coefficient = BASE_UPDATE_COEFFICIENT * (1/ math.log(new_gp+1)) 
        
        diff = (encoded_game_val - embeddings[player]['current'])#*(mp[player][game_id]/48)
        dist = torch.norm(diff)
        counter += 1
        if counter > 28000:
            loss_embeddings += dist**2
        
        embeddings[player]['current'] += update_coefficient * diff 
        embeddings[player][game_id] = copy.deepcopy(embeddings[player]['current'])
        old_game_id = game_id
            
    print(loss_embeddings.item())
    return embeddings, gp

**GETS TEAM AFFILIATIONS

In [11]:
def get_nearest_player(player, embeddings):
    
    def has_funky_characters(name):
        for character in name:
            if ord(character) > ord("z"):
                return True
        return False
    
    HIST_LENGTH = ord("z")
    def create_letter_histogram(source):
        letter_histogram = np.zeros(HIST_LENGTH)
        for letter in source:
            if ord(letter) < HIST_LENGTH:
                letter_histogram[ord(letter)] += 1
        return letter_histogram
    
    def get_difference(hist1, hist2):
        return sum([abs(hist2[i] - hist1[i]) for i in range(HIST_LENGTH)])
    
    name = get_player_name(player)
    if name in embeddings:
        return name
    if name.replace("Jr.", "").rstrip() in embeddings:
        return name.replace("Jr.", "").rstrip()
    
    reference = create_letter_histogram(name)
    
    closest_name = ""
    best_match_score = 100
    for candidate in embeddings:
        score = get_difference(reference, create_letter_histogram(candidate))
        if score < best_match_score:
            best_match_score = score
            closest_name = candidate
            
    return closest_name if has_funky_characters(closest_name) else name

def update_minutes_played(mpval, mp, embeddings):

    MP_VAL_PLAYER_IDX = 0
    MP_VAL_MINUTES_IDX = 1

    for mp_value in mpval:
        player = get_player_name(mp_value[MP_VAL_PLAYER_IDX])
        if player not in mp:
            player = get_nearest_player(player, embeddings)
        if player in mp:
            mp[player]['projected']= mp_value[MP_VAL_MINUTES_IDX]

    for player in injured_players:
        mp[player]["projected"] = 0

    for team in NBA:
        roster = [get_nearest_player(get_player_name(name),embeddings) for name in filter(lambda name: get_nearest_player(get_player_name(name),embeddings) in mp, heightweight[team][2021])]
        total_minutes_played = sum([mp[player]['projected'] for player in roster])
        for player in roster:
            mp[player]['projected'] /= (total_minutes_played / TOTAL_TEAM_MINUTES)
    
    return mp

def initialize_minutes_played(embeddings):
    mp = {}
    for player_name in embeddings:
        mp[player_name] = {"projected": 0}
    for game in gamescores:
        player_name = get_nearest_player(get_player_name(game[PLAYER_IDX]), embeddings)
        mp[player_name][game[GAME_ID_IDX]] = game[MINUTES_IDX]
    return mp

def create_minutes_played(embeddings):
    mp = initialize_minutes_played(embeddings)
    return update_minutes_played(mpval, mp, embeddings)


In [12]:
def create_affiliations(embeddings):
    affiliations = {}
    for i, game in enumerate(gamescores):
        player = get_nearest_player(get_player_name(game[PLAYER_IDX]), embeddings)
        if player not in affiliations:
            affiliations[player] = {}
        team = game[TEAM_IDX]
        if team in team_to_abbr_mapping:
            if game[GAME_ID_IDX] in affiliations[player]:
                print(GAME_ID_IDX, player)
            affiliations[player][game[GAME_ID_IDX]] = team
    return affiliations

**PREDICTING CHANGES IN EMBEDDINGS FROM YEAR TO YEAR

In [13]:
#PREDICT YEAR-TO-YEAR IMPROVEMENTS
#now trying to predict scores embeddings
class EmbeddingAdjustmentModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.__layer_one = nn.Linear(8, 4)
        self.__layer_two = nn.Linear(4, 4)
    
    def forward(self, this_year):
        return self.__layer_two(F.gelu(self.__layer_one(this_year)))

In [14]:
class YearToYearImprovementDataset(data.Dataset):
        
    def __init__(self, yearlist, embeddings):
        super().__init__()
        EPSILON = 10e-4
        
        x_list = []
        y_list = []
        for year in yearlist:
            for player in filter(lambda name: str(year) in embeddings[name] and str(year + 1) in embeddings[name] and name in mp, embeddings):
                team = YearToYearImprovementDataset.find_team(player, year)
                current = YearToYearImprovementDataset.construct_vector_representation(team, player, year, embeddings)
                target = self.__get_target(player, year, embeddings)
                if not isinstance(current, type(None)) and np.linalg.norm(target.numpy()) > EPSILON:
                    x_list.append(current)
                    y_list.append(target)
        self.__x = torch.stack(x_list).double()
        self.__y = torch.stack(y_list).double()
    
    def __getitem__(self, idx):
        return (self.__x[idx], self.__y[idx])
    
    def __len__(self):
        return len(self.__x) 
    
    @staticmethod
    def construct_vector_representation(team, player, year, embeddings):
        skills = embeddings[player][str(year)]
        if team != None:
            height = int(heightweight[team][year][player]['height'])
            weight = int(heightweight[team][year][player]['weight'])
            dob = heightweight[team][year][player]['age']
            age = YearToYearImprovementDataset.get_age(dob, year)
            draft_pos = draft[player][1] if player in draft else 61
            descriptors = torch.tensor(np.asarray([height, weight, age, draft_pos])).float()
            overall_vec = torch.cat([skills, descriptors], dim=0).detach()
            return overall_vec.double()
        return None
    
    @staticmethod
    def get_age(date_of_birth, current_year):
        yob = int(re.search('\d{4}', date_of_birth).group(0))
        return current_year - yob
    
    @staticmethod
    def find_team(player, year):
        for team in heightweight:
            if player in heightweight[team][year]:
                return team
        return None
    
    def __get_target(self, player, year, embeddings):
        return embeddings[player][str(year + 1)] - embeddings[player][str(year)]

In [15]:
def create_year_adjustment_datasets(embeddings):
    return {
        "overall": YearToYearImprovementDataset(range(2010, 2020), embeddings),
        "training": YearToYearImprovementDataset(range(2010, 2019), embeddings),
        "validation": YearToYearImprovementDataset(range(2019, 2020), embeddings)
    }

def train_player_model(model_players, datasets, use_all=False, num_epochs=200, batch_size=64):
    optimizer_players = optim.AdamW(model_players.parameters(), lr=1e-4) # change the learning rate if you'd like
    with torch.enable_grad():
        run_through_model(model_players, datasets["training"] if not use_all else datasets["overall"], batch_size=batch_size, num_epochs=num_epochs, optimizer=optimizer_players, regularize=True)

def validate_player_model(model_players, datasets):
    with torch.no_grad():
        run_through_model(model_players, datasets["validation"], batch_size=1, num_epochs=1)

def create_and_train_year_adjustment_model(datasets, use_all=False, num_epochs=200, batch_size=64):
    model_players = EmbeddingAdjustmentModel().double()
    train_player_model(model_players, datasets, use_all=use_all, num_epochs=num_epochs, batch_size=batch_size)
    if not use_all:
        print("validating")
        validate_player_model(model_players, datasets)
    return model_players

In [16]:
def update_embeddings_with_predictions(model_players, embeddings, coeff=1):
    for player in filter(lambda name: torch.max(abs(embeddings[name]["2020"])) > 0 and mp[name]["projected"] > 0, embeddings):
        skills = embeddings[player]['2020']
        team = YearToYearImprovementDataset.find_team(player, 2020)
        vector_input = YearToYearImprovementDataset.construct_vector_representation(team, player, 2020, embeddings)
        if not isinstance(vector_input, type(None)):
            with torch.no_grad():
                prediction = model_players(vector_input)
            embeddings[player]['predicted'] = (prediction * coeff) + embeddings[player]['2020']
    return embeddings

**CREATING DATA, DEFININING, AND TRAINING MODEL FOR CONVERTING PLAYER EMBEDDINGS TO TEAM EMBEDDINGS TO WINS

In [17]:
class TeamDataset:
    
    def __init__(self, games, affiliations, batch_size=1):
        super().__init__()
        self.__winners = []
        self.__losers = []
        for (game_id, winner, loser) in filter(lambda x: len(x[0]) >= 3, games):
            self.__winners.append(TeamDataset.create_team_tensor(winner, game_id, affiliations))
            self.__losers.append(TeamDataset.create_team_tensor(loser, game_id, affiliations))
            if isinstance(self.__winners[-1], type(None)) or isinstance(self.__losers[-1], type(None)):
                self.__winners = self.__winners[:-1]
                self.__losers = self.__losers[:-1]
        self.__order = self.__generate_random_order()
        self.__idx = 0
        self.__batch_size = batch_size
    
    def __getitem__(self, idx):
        return (self.__winners[idx], self.__losers[idx])
    
    def __len__(self):
        return len(self.__winners)
    
    def __iter__(self):
        self.__idx = 0
        self.__order = self.__generate_random_order()
        return self
    
    def __next__(self):
        if self.__idx + self.__batch_size > len(self):
            raise StopIteration
        value = self[self.__idx:self.__idx + self.__batch_size]
        self.__idx += self.__batch_size
        return value
    
    def __generate_random_order(self):
        return random.choices(range(len(self)), k=len(self))
        
    def set_batch_size(self, batch_size):
        self.__batch_size = batch_size
    
    @staticmethod
    def create_team_tensor(team, game_id, affiliations):
        team_list = []
        for player in filter(lambda name: game_id in embeddings[name] and game_id in affiliations[name], embeddings):
            if affiliations[player][game_id] == team and mp[player][game_id] > 0:
                minutes_tensor = torch.tensor(mp[player][game_id], dtype=torch.float32)
                embedding_plus_minutes = torch.cat([embeddings[player][game_id], minutes_tensor.unsqueeze(0)], dim=0)
                team_list.append(embedding_plus_minutes.float().detach())
        return torch.stack(team_list) if len(team_list) > 0 else None

In [18]:
class PlayerCombinerModel(nn.Module):
        
    def __init__(self, just_minutes=True):
        super().__init__()
        
        EMBEDDING_SIZE = 4
        self.__player_layer = nn.Linear(EMBEDDING_SIZE + 1, 1) 
        #self.__team_layer_one = nn.Linear(EMBEDDING_SIZE * 2, 4)
        self.__team_layer_one = nn.Linear(EMBEDDING_SIZE, 4)
        self.__team_layer_two = nn.Linear(4, 1)
        self.__just_minutes = just_minutes
    
    def forward(self, team_one_player_lists, team_two_player_lists):
        
        outputs = []
        for team_one_player_list, team_two_player_list in zip(team_one_player_lists, team_two_player_lists):
            team_one_embedding = self.find_team_embedding(team_one_player_list)
            team_two_embedding = self.find_team_embedding(team_two_player_list)
            outputs.append(self.score_embeddings(team_one_embedding, team_two_embedding))
            
        return torch.stack(outputs)
    
    def find_team_embedding(self, tensor_list):
        if not self.__just_minutes:
            weights = self.__player_layer(tensor_list)
            normalized_weights = F.softmax(weights, dim=0)
        else:
            weights = tensor_list[:, -1].unsqueeze(1)
            normalized_weights = weights / torch.sum(weights)
        return torch.sum((normalized_weights * tensor_list[:, 0:4]), dim=0).squeeze()
    
    def score_embeddings(self, team_one_embedding, team_two_embedding):
        def score(combined):
            return self.__team_layer_two(torch.tanh(self.__team_layer_one(combined)))
        #output_one = score(torch.cat([team_one_embedding, team_two_embedding], dim=0))
        #output_two = score(torch.cat([team_two_embedding, team_one_embedding], dim=0))
        output_one = score(team_one_embedding)
        output_two = score(team_two_embedding)
        
        return torch.softmax(torch.cat([output_one, output_two], dim=0), dim=0)

In [19]:
def create_team_data_sets(gameresults, affiliations):
    return {
        "overall": TeamDataset(gameresults, affiliations),
        "training": TeamDataset(gameresults[0:int(0.8 * len(gameresults))], affiliations),
        "validation": TeamDataset(gameresults[int(0.8 * len(gameresults)):], affiliations)
    }

def run_through_pc_model(pc_model, dataset, batch_size, num_epochs, optimizer=None):
    criterion = nn.MSELoss()
    dataset.set_batch_size(batch_size)
    for epoch in range(num_epochs):
        total_loss = 0
        total_prediction = 0
        for (winners, losers) in dataset:
            prediction = pc_model(winners, losers)
            loss = criterion(prediction[:, 0].double(), torch.ones(batch_size).double())
            total_loss += loss.item()
            total_prediction += (torch.sum(prediction[:, 0]).item() / batch_size)
            if optimizer != None:
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
        
        num_runs = int(len(dataset) / batch_size)
        print(total_loss / num_runs, total_prediction / num_runs)

def train_team_model(pc_model, datasets, use_all=False, num_epochs=20, batch_size=32):
    pc_optimizer = optim.AdamW(pc_model.parameters(), lr=1e-3)
    with torch.enable_grad():
        run_through_pc_model(pc_model, datasets["training"] if not use_all else datasets["overall"], batch_size=batch_size, num_epochs=num_epochs, optimizer=pc_optimizer)
    return pc_model
    
def validate_team_model(pc_model, datasets):
    with torch.no_grad():
        run_through_pc_model(pc_model, datasets["validation"], batch_size=1, num_epochs=1)

def create_and_train_team_model(datasets, use_all=False, num_epochs=12, batch_size=64, just_minutes=False):
    pc_model = PlayerCombinerModel(just_minutes)
    pc_model = train_team_model(pc_model, datasets, use_all=use_all, num_epochs=num_epochs, batch_size=batch_size)
    if not use_all:
        validate_team_model(pc_model, datasets)
    return pc_model

In [20]:
def create_team_to_player_map(embeddings, heightweight, mp):
    team_to_player_map = {}
    player_to_team_map = {}
    for team in heightweight:
        team_to_player_map[team] = []
        for player in heightweight[team][2021]:
            player_name = get_nearest_player(get_player_name(player), embeddings)
            if player_name in mp:
                if mp[player_name]["projected"] > 0: #MIN_MINUTES instead of 0
                    team_to_player_map[team].append(player_name)
                    if player_name in player_to_team_map:
                        print(player_name, player_to_team_map[player_name], player)
                    player_to_team_map[player_name] = (team, player)
    return team_to_player_map, player_to_team_map

In [21]:
def create_team_abilities(pc_model, heightweight, embeddings, mp, team_to_player_map, value='predicted'):
    team_abilities = {}
    for team in team_to_player_map:
        team_list = []
        for player in team_to_player_map[team]:
            player_name = get_nearest_player(get_player_name(player), embeddings)
            if mp[player_name]['projected'] > 0:
                minutes_tensor = torch.tensor(mp[player_name]['projected'], dtype=torch.float32)
                predicted_score = embeddings[player_name][value].double().squeeze() if value in embeddings[player_name] else embeddings[player_name]['2020'].double()
                embedding_plus_minutes = torch.cat([predicted_score, minutes_tensor.unsqueeze(0).double()], dim=0)
                team_list.append(embedding_plus_minutes.float().detach())
        team_tensor = torch.stack(team_list)
        with torch.no_grad():
            team_abilities[team] = pc_model.find_team_embedding(team_tensor)
    return team_abilities

In [22]:
def create_team_win_probability_matrix(pc_model, team_abilities):
    team_win_prob_matrix = {}
    for team1 in NBA:
        if team1 not in team_win_prob_matrix:
            team_win_prob_matrix[team1] = {}
        for team2 in filter(lambda name: name != team1 and name not in team_win_prob_matrix[team1], NBA):
            if team2 not in team_win_prob_matrix:
                team_win_prob_matrix[team2] = {}
            with torch.no_grad():
                prediction = pc_model.score_embeddings(team_abilities[team1], team_abilities[team2])
            team_win_prob_matrix[team1][team2] = prediction[0].item()
            team_win_prob_matrix[team2][team1] = prediction[1].item()
    return team_win_prob_matrix

**FINDING EXPECTED NUMBER OF WINS

In [23]:
def create_potential_matchups_and_record_maps(include_season_to_date=False):
    record = {}
    potential_matchups = {}
    for team in NBA:
        if team not in potential_matchups:
            potential_matchups[team] = {}
            record[team] = {"W": 0, "L": 0}
        for opponent in filter(lambda name: name != team, NBA):
            if opponent not in potential_matchups:
                potential_matchups[opponent] = {}
                record[opponent] = {"W": 0, "L": 0}
            number_of_matchups = 3 if ((team in East and opponent in East) or (team in West and opponent in West)) else 2
            potential_matchups[team][opponent] = {"played": 0, "max": number_of_matchups}
            potential_matchups[opponent][team] = {"played": 0, "max": number_of_matchups}
    if include_season_to_date:
        update_standings_with_current_season(gameresults, potential_matchups, record)
 
    return potential_matchups, record

In [24]:
# SKIP THIS IF YOU WANT TO RUN FROM START OF SEASON
def update_standings_with_current_season(gameresults, potential_matchups, record):
    CURRENT_SEASON_IDX = 14033
    current_season = gameresults[CURRENT_SEASON_IDX:len(gameresults), :]
    for (game_id, winner, loser) in current_season:

        winner_name = team_to_abbr_mapping[winner]
        loser_name = team_to_abbr_mapping[loser]
        potential_matchups[winner_name][loser_name]["played"] += 1
        potential_matchups[loser_name][winner_name]["played"] += 1

        if winner_name not in record:
            record[winner_name] = {"W": 0, "L": 0}
        record[winner_name]["W"] += 1

        if loser_name not in record:
            record[loser_name] = {"W": 0, "L": 0}
        record[loser_name]["L"] += 1
    return record

In [25]:
def find_expected_wins(gameresults=None, include_season_to_date=False, display=True):
    potential_matchups, record = create_potential_matchups_and_record_maps(include_season_to_date=include_season_to_date)
    if include_season_to_date:
        update_standings_with_current_season(gameresults, potential_matchups, record)
    for team1 in NBA:
        for team2 in filter(lambda x: x != team1, NBA):
            matchups_to_simulate = potential_matchups[team1][team2]["max"] -potential_matchups[team1][team2]["played"]
            for matchup in range(matchups_to_simulate):
                potential_matchups[team1][team2]["played"] += 1
                potential_matchups[team2][team1]["played"] += 1

                prediction = team_win_prob_matrix[team1][team2]
                record[team1]["W"] += prediction
                record[team1]["L"] += 1 - prediction
                record[team2]["W"] += 1 - prediction
                record[team2]["L"] += prediction
    
    sorted_team_list = sorted([(team, record[team]["W"]) for team in record], key=lambda x: x[1], reverse=True)
    if display:
        for i, (team, wins) in enumerate(sorted_team_list):
            print(i + 1, team, round(wins,2))
    return sorted_team_list

**FINDING BEST PLAYERS CURRENTLY

In [26]:
def find_best_current_players(field, box_score_model, display=True):
    with torch.no_grad():
        result_list = [(player, convert_war(box_score_model.predict(embeddings[player][field].double()).item())) for player in filter(lambda x: field in embeddings[x], embeddings)]
    if display:
        for i, (player, value) in enumerate(sorted(filter(lambda x: gp[x[0]] > 50, result_list), key=lambda x: x[1], reverse=True)):
            print(i + 1, player, value)
    return result_list
    

In [720]:
def convert_war(plus_minus, ind_game=True):
    if ind_game == False:
        points_added = plus_minus*36
    else:
        points_added = plus_minus
    points_base = math.floor(points_added)
    remainder = points_added-points_base
    if points_base>0:
        wins_added = (1-remainder)*cdf[points_base-1,1]+remainder*cdf[points_base,1]
    elif points_base<0:
        points_lost = abs(points_base)
        wins_added = -1*((1-remainder)*cdf[points_lost-1,1]+remainder*cdf[points_lost,1])
    else:
        wins_added = (1-remainder)*cdf[points_base,1]
    wins_added /= 2
    wins_added = round(wins_added,3)
    if wins_added == -0.5:
        print(plus_minus)
    return wins_added

In [28]:
def convert_war_game(gameperf):
    points_added = gameperf

In [29]:
def player_history(player):
    history = {}
    for game in embeddings[player]:
        if game != "current" and game != "predicted":
            history[game]=convert_war(box_score_model.predict(embeddings[player][game].double()).item())
    df_h = pd.DataFrame(list(history.items()),columns = ['game','skill']) 
    address = "history/"+player+".csv"
    df_h.to_csv(address)


**SIMULATING SEASON

In [30]:
def simulate_game(team_win_prob_matrix, team_one, team_two):
    if random.random() < team_win_prob_matrix[team_one][team_two]:
        return True, team_one, team_two
    else:
        return False, team_two, team_one

In [31]:
def get_seed(conference, sorted_teams):
    return [team for team in filter(lambda name: name in conference, sorted_teams)][:10]

In [32]:
def get_seed_index(seed):
    return seed-1

In [33]:
def playin_games(team_win_prob_matrix, seeds): 
    
    def get_team(seed_number):
        return seeds[get_seed_index(seed_number)]
    
    _, advancingseed_seven, loserseed = simulate_game(team_win_prob_matrix, get_team(7), get_team(8))    
    _, still_eligible_seed, _ = simulate_game(team_win_prob_matrix, get_team(9), get_team(10))
    _, advancingseed_eight, _ = simulate_game(team_win_prob_matrix, loserseed, still_eligible_seed)
    return advancingseed_seven, advancingseed_eight

In [34]:
def matchup(team_win_prob_matrix, teamone, teamtwo):
    SERIES_LENGTH = 7
    GAMES_NEEDED_TO_WIN = 4
    team_one_wins = 0
    for game in range(SERIES_LENGTH):
        team_one_won, _, _ = simulate_game(team_win_prob_matrix, teamone, teamtwo)
        team_one_wins = team_one_wins + 1 if team_one_won else team_one_wins
    return teamone if team_one_wins >= GAMES_NEEDED_TO_WIN else teamtwo

In [35]:
def playoff_games(team_win_prob_matrix, seeds):
    
    def get_team(seed_number):
        return seeds[get_seed_index(seed_number)]
    
    semifinalist_one = matchup(team_win_prob_matrix, get_team(1), get_team(8))
    semifinalist_two = matchup(team_win_prob_matrix, get_team(2), get_team(7))
    semifinalist_three = matchup(team_win_prob_matrix, get_team(3), get_team(6))
    semifinalist_four = matchup(team_win_prob_matrix, get_team(4), get_team(5))
    
    finalist_one = matchup(team_win_prob_matrix, semifinalist_one, semifinalist_four)
    finalist_two = matchup(team_win_prob_matrix, semifinalist_two, semifinalist_three)
    
    return matchup(team_win_prob_matrix, finalist_one, finalist_two)

In [36]:
def get_seeds(conference, record):
    random.shuffle(conference) # shuffled so that ties are broken randomly instead of alphabetically
    return sorted(conference, key=lambda team:record[team]["W"], reverse=True)

In [37]:
def run_conference_playoffs(team_win_prob_matrix, conference, record):
    seeds = get_seeds(conference, record)
    
    seven_seed, eight_seed = playin_games(team_win_prob_matrix, seeds)
    seeds[get_seed_index(7)] = seven_seed
    seeds[get_seed_index(8)] = eight_seed
    
    return playoff_games(team_win_prob_matrix, seeds), seeds[0:8]

In [38]:
def run_playoffs(team_win_prob_matrix, record):
    eastern_finalist, eastern_playoff_teams = run_conference_playoffs(team_win_prob_matrix, East, record)
    western_finalist, western_playoff_teams = run_conference_playoffs(team_win_prob_matrix, West, record)
    winner = matchup(team_win_prob_matrix, eastern_finalist, western_finalist)
    return {"winner": winner,
            "finalists": [eastern_finalist, western_finalist],
            "playoffs": eastern_playoff_teams + western_playoff_teams,
           }

In [39]:
def simulate_regular_season(team_win_prob_matrix, include_season_to_date=True):
    potential_matchups, record = create_potential_matchups_and_record_maps(include_season_to_date=include_season_to_date)
    for team1 in NBA:
        for team2 in filter(lambda x: x != team1, NBA):
            matchups_to_simulate = potential_matchups[team1][team2]["max"] - potential_matchups[team1][team2]["played"]
            for matchup in range(matchups_to_simulate):
                _, winner, loser = simulate_game(team_win_prob_matrix, team1, team2)
                record[winner]["W"] += 1
                record[loser]["L"] += 1
                potential_matchups[team1][team2]["played"] += 1
                potential_matchups[team2][team1]["played"] += 1
    return record

In [40]:
def simulate_season(team_win_prob_matrix, include_season_to_date=True):
    record = simulate_regular_season(team_win_prob_matrix, include_season_to_date)
    return run_playoffs(team_win_prob_matrix, record)

In [41]:
def simulate_seasons(team_win_prob_matrix, num_simulations=100, adjust=True, include_season_to_date=True):
    playoffs={}
    finals={}
    champions={}

    for team in NBA:
        playoffs[team] = 0
        finals[team] = 0
        champions[team] = 0

    for i in range(num_simulations):
        results = simulate_season(team_win_prob_matrix, include_season_to_date)
        champions[results["winner"]] += 1
        for team in results["finalists"]:
            finals[team] += 1
        for team in results["playoffs"]:
            playoffs[team] += 1

    if adjust:
        for team in NBA:
            playoffs[team] /= num_simulations
            finals[team] /= num_simulations
            champions[team] /= num_simulations
    
    return champions, finals, playoffs

In [42]:
def sort_results(results):
    sorted_list = sorted([(team, results[team]) for team in playoffs], key=lambda x: results[x[0]], reverse=True)
    for i, (team, prob) in enumerate(sorted_list):
        print(i + 1, team, prob)
    return sorted_list

In [43]:
class MinutesProjectionModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        INPUT_SIZE = 4
        NUM_POSITIONS = 5
        self.__player_layer = nn.Linear(INPUT_SIZE, NUM_POSITIONS).float()
        self.__minutes_layer = nn.Linear(INPUT_SIZE, 1).float()
    
    def forward(self, player_list, allowed_positions):
        
        team_projections = []
        minutes_projections = []
        counter = 0
        for i, player_vec in enumerate(player_list):
            player_vec_to_use = player_vec.float().detach()
            player_minutes_projection_init = F.softmax(self.__player_layer(player_vec_to_use), dim=0)
            positions_mask = torch.from_numpy(np.array(allowed_positions[i])).detach().float()
            player_minutes_projection_allowed = player_minutes_projection_init*positions_mask
            team_projections.append(player_minutes_projection_allowed)
            
            minutes_projection = self.__minutes_layer(player_vec_to_use).squeeze()
            minutes_projections.append(minutes_projection)

                    
        # standardize minutes projections so that the total minutes played by each player
        minutes_vector = torch.stack(minutes_projections).float()
        standardized_minutes_vector = F.softmax(minutes_vector, dim=0).float() * torch.tensor([240]).squeeze().float().detach()
        
        # get minutes at each position
        tot_min_list = []
        for i in range(len(player_list)):
            total_minutes_for_player = standardized_minutes_vector[i]*team_projections[i] #total minutes for that player
            tot_min_list.append(total_minutes_for_player)

            
        team_stack = torch.stack(tot_min_list) #team_stack is nx5
        positions_sum_vec_with_zeroes = torch.sum(team_stack, dim=0).float()
        
        
        #in case any are zero
        EPSILON = torch.tensor([1e-5]).squeeze().float().detach()
        positions_sum_vec = positions_sum_vec_with_zeroes + EPSILON
    
        total_minutes_for_player_final = []
        for i in range(len(player_list)):
            total_minutes_for_player_final.append(tot_min_list[i] / positions_sum_vec)

        output_matrix=torch.stack(total_minutes_for_player_final)
        
        total_min_output = []
        for i in range(len(output_matrix)):
            total_min_output.append(torch.sum(output_matrix[i]))
        
        
        final_output = torch.stack(total_min_output)

        #final_output = torch.from_numpy(np.array(total_min_output,dtype="float64"))
        
        return final_output


In [ ]:

#input mindf[___,2] [also age, ability, draft position] together as a team
#intermediate output: predict nx5 dimensional matrix (minutes by position by player)
#L1 norm to zero out marginal players
#softmax to get each column between 0 and 1
#FINAL output: predict mindf[__,4] from mindf[__,2]

mindict=create_minute_dictionary()
min_in, min_out, min_mask = get_team_minute_info("CHI",2018)

criterion = nn.MSELoss()
test_model = MinutesProjectionModel().float()
minutes_optimizer = optim.AdamW(test_model.parameters(), lr=1e-1)


total_loss = 0
#with torch.enable_grad():

result = test_model(min_in,min_out)
print(result.type())
print(wasout)
loss = criterion(result, wasout)
total_loss += loss.item()
print(loss)
if minutes_optimizer != None:
    loss.backward()
    minutes_optimizer.step()
    minutes_optimizer.zero_grad()

In [ ]:
for name, parameter in test_model.named_parameters():
    print(name, parameter.requires_grad)

In [45]:
#JUMPO
def create_minute_dictionary():
    mindict = {}
    for i in range(len(mindf)):
        name = mindf[i,0]
        year0 = mindf[i,1]
        year1 = mindf[i,3]
        if name in mindict:
            mindict[name][year1] = mindf[i,4]
        else:
            mindict[name]={year0: mindf[i,2], year1: mindf[i,4]}
        mindict[name]['mask'] = mindf[i,5:10]
    return(mindict)

def get_player_min_inputs(player, team, year):
    roster = heightweight[team][year]
    player_name = get_nearest_player(get_player_name(player), embeddings)
    dob = roster[player]['age']
    age = YearToYearImprovementDataset.get_age(dob, year)
    draft_pos = draft[player][1] if player in draft else 61
    skill=box_score_model.predict(embeddings[player_name][str(year)].double()).item()
    
    prev_year = year - 1
    if player_name not in mindict:
        min_pre = 0
    elif prev_year in mindict[player_name]:
        min_pre = mindict[player_name][year-1]
    else:
        min_pre = 0
        
    descriptors = torch.tensor(np.asarray([age, draft_pos, skill, min_pre])).float()
    full_input = torch.cat([descriptors], dim=0).detach()
    return(full_input)

def get_team_minute_info(team,year):
    xlist=[]
    ylist=[]
    masklist = []
    roster = heightweight[team][year]

    for player in roster:
        player_name = get_nearest_player(get_player_name(player), embeddings)
        new_input = get_player_min_inputs(player,team,year)
        if player_name in mindict:
            if year in mindict[player_name]:
                new_target = mindict[player_name][year]
            else:
                new_target = 0
        
        if player_name in mindict:    
            masklist.append(mindict[player_name]['mask'])
            xlist.append(new_input)
            ylist.append(new_target)
            
    #modify ylist
    tot_min = 0
    for i in range(len(ylist)):
        tot_min += ylist[i]
    for i in range(len(ylist)):
        ylist[i] /= (tot_min/240)
    ylist = torch.from_numpy(np.array(ylist, dtype="float64"))
    
    return xlist, ylist, masklist

In [597]:
def get_best_rookies(keep_print=True, year = 2020):    
    draft_results={}
    for player in draft:
        if draft[player][0]==year:
            if player in embeddings:
                win_draft=convert_war(box_score_model.predict(embeddings[player]['current'].double()).item())
                draft_results[player]=win_draft
    sorted_list = sorted([(player, draft_results[player]) for player in draft_results], key=lambda x: draft_results[x[0]], reverse=True)
    if keep_print==True:    
        for i, (player, wins) in enumerate(sorted_list):
            if i<10 and wins > 0.1:
                print(i + 1, player, wins)
    return sorted_list

def rookie_of_the_year(sorted_list_mvp, keep_print=True, year = 2020):
    roty_team = []
    taken = 0
    roty = 'None'
    for playerperformance in sorted_list_mvp:
        player = playerperformance[0]
        performance = playerperformance[1]
        if player in draft:
            draft_year = draft[player][0]
            if draft_year == year:
                if keep_print == True:
                    print(player,performance)
                if taken == 0:
                    roty = player
                if taken < 5:
                    roty_team.append(player)
                taken+=1
    return roty, roty_team

In [446]:
#fix trades
def add_inseason_trades():
    heightweight['CLE'][2021]['Jarrett Allen'] = heightweight['BRK'][2021]['Jarrett Allen']
    heightweight['CLE'][2021]['Taurean Prince'] = heightweight['BRK'][2021]['Taurean Prince']
    heightweight['HOU'][2021]['Victor Oladipo'] = heightweight['IND'][2021]['Victor Oladipo']
    heightweight['IND'][2021]['Caris LeVert'] = heightweight['BRK'][2021]['Caris LeVert']
    heightweight['BRK'][2021]['James Harden'] = heightweight['HOU'][2021]['James Harden']
    
    del heightweight['BRK'][2021]['Jarrett Allen']
    del heightweight['BRK'][2021]['Taurean Prince']
    del heightweight['IND'][2021]['Victor Oladipo']
    del heightweight['BRK'][2021]['Caris LeVert']
    del heightweight['HOU'][2021]['James Harden']
    
    mp = create_minutes_played(embeddings)
    
    return heightweight, mp
    

# **CALCULATING PERFORMANCE DURING YEAR TO DATE

In [773]:
#INSEASON JUMP HERE

IDX_2010 = range(0,1170)
IDX_2011 = range(1312,2543)
IDX_2012 = range(2624,3559)
IDX_2013 = range(3696,4926)
IDX_2014 = range(5010,6241)
IDX_2015 = range(6329,7559)
IDX_2016 = range(7640,8871)
IDX_2017 = range(8956,10187)
IDX_2018 = range(10266,11496)
IDX_2019 = range(11577,12808)
IDX_2020 = range(12889,13949)
IDX_2021 = range(14032,len(gameresults))
    
def season_leaders(range_val, keep_print = True):
    war = {}
    ind_game_war = []
    
    curr_season = gameresults[range_val,:] 
    list_game_results = list([result[0] for result in gameresults])

    for i in range(len(gamescores)):
        game = gamescores[i,40]
        if game in curr_season[:,0]:
            player=gamescores[i,0]
            war[player]=0

    
    for game in gamescores:
        game
        
        
    marked = set()
    with torch.no_grad():
        for i in range(len(gamescores)): 
            game = gamescores[i,40]
            player=gamescores[i,0]
            if game in curr_season[:,0] and (game, player) not in marked and gamescores[i, -1] > 0:

                marked.add((game, player))
                gameval=gamescores[i,keep].astype('double')
                gamevalinput=torch.from_numpy(gameval)

                with torch.no_grad():
                    plus_minus = (box_score_model(gamevalinput).item() * gamescores[i, -1])
                    wins_added = convert_war(plus_minus)

                war[player] += wins_added
                ind_game_war.append((player, game, plus_minus))

                

    if keep_print == True:
        sorted_list_curr = sorted([(player, war[player]) for player in war], key=lambda x: war[x[0]], reverse=True)
        counter = 0
        for i, (player, wins) in enumerate(sorted_list_curr):
            if mp[get_nearest_player(player,mp)]['projected']>0:
                counter +=1
                player_name = get_plain(player)
                print(counter,"\t", player_name, "\t", round(wins, 3))
            
        
    return war, ind_game_war
    

In [651]:
def get_all_nba(leaderboard):
    positions = get_positions()
    all_nba_pos = {}
    for teamnum in ['first','second','third']:
        all_nba_pos[teamnum]={'guard': 0, 'forward': 0, 'c': 0}

    all_nba=[]
    num_players={'first': 0, 'second': 0, 'third': 0}


    for i in range(25):
        player = get_nearest_player(leaderboard[i][0],positions)
        pos_vec = positions[player]
        pos_sum = 0
        if pos_vec['pg']==1 or pos_vec['sg']==1:
            pos_vec['guard']=1
        if pos_vec['sf']==1 or pos_vec['pf']==1:
            pos_vec['forward']=1
        for position in ['guard','forward','c']:
            if position in pos_vec:
                pos_sum+=pos_vec[position]
        for position in ['guard','forward','c']:
            if position in pos_vec:
                pos_vec[position]/=pos_sum

        placed = 0
        for teamnum in ['first','second','third']:
            available_pos = 0
            if placed == 0:
                for position in all_nba_pos[teamnum]:
                    if position == 'c':
                        idx = 1
                    else:
                        idx = 2
                    if all_nba_pos[teamnum][position]<idx and position in pos_vec:
                        available_pos += 1
                        all_nba_pos[teamnum][position]+=pos_vec[position]   
                if available_pos > 0 and num_players[teamnum]<5:
                    player = get_plain(player)
                    all_nba.append([player, teamnum])
                    placed += 1
                    num_players[teamnum] += 1

    first_team = []
    second_team = []
    third_team = []
    for i in range(15):
        if all_nba[i][1]=="first":
            first_team.append(all_nba[i][0])
        elif all_nba[i][1]=="second":
            second_team.append(all_nba[i][0])
        else:
            third_team.append(all_nba[i][0])
    return first_team, second_team, third_team

def most_improved_player(keep_print=True, field_curr = 'current', field_base = '2020', field_mp = 'projected'):
    team_improv = {}
    player_improv = {}
    for team in NBA:    
        skilltot = 0
        skilltot_prior=0
        for player in team_to_player_map[team]:
            player_name = get_nearest_player(get_player_name(player), embeddings)
            if field_mp in mp[player_name] and mp[player_name][field_mp] > 0:
                    minutes_tensor = torch.tensor(mp[player_name][field_mp], dtype=torch.float32)
                    predicted_plusminus = box_score_model.predict(embeddings[player_name][field_curr].double())*minutes_tensor
                    val = round(predicted_plusminus.item(),1)
                    skilltot += val
                    team_list.append(predicted_plusminus.float().detach())
                    prior_plusminus = box_score_model.predict(embeddings[player_name][field_base].double())*minutes_tensor
                    val_prior = round(prior_plusminus.item(),1)
                    skilltot_prior += val_prior
                    change = round(val - val_prior,1)
                    player_improv[player] = {'current': val, 'start': val_prior, 'change': change}

        change_team = round(skilltot - skilltot_prior,1)
        team_improv[team]={'current': skilltot, 'start': skilltot_prior, 'change': change_team}
    
    
    sorted_list = sorted([(player, player_improv[player]['change']) for player in player_improv], key = lambda x: x[1], reverse=True)
    
    if keep_print==True:
        for i, (player, improvement) in enumerate(sorted_list):
            print(i + 1, player, improvement) 

    return sorted_list

def get_all_stars():
    num_east = 0
    num_west = 0
    east_allstars = []
    west_allstars = []
    for i in range(50):
        player = get_nearest_player(leaderboard[i][0],player_to_team_map)   
        team = player_to_team_map[player][0]
        if team in East:
            if num_east < 12:
                num_east += 1
                player = get_plain(player)
                east_allstars.append(player)
        else:
            if num_west < 12:
                num_west += 1
                player = get_plain(player)
                west_allstars.append(player)
    return east_allstars, west_allstars

def get_best_game(keep_print=True, num_print=25, ind_game_war = ind_game_war):
    
    indgameres = {}
    for game in ind_game_war:
        uniqid = game[0] + " " + game[1]
        indgameres[uniqid] = game[2]
    sorted_list_games = sorted([(player, indgameres[player]) for player in indgameres], key=lambda x: indgameres[x[0]], reverse=True)
    if keep_print == True:
        for i, (game, points) in enumerate(sorted_list_games):
            if(i<num_print):
                print(i+1,"\t", game, "\t", round(points, 1))
            
    return sorted_list_games

def get_historical_minute_averages(mp): 
    avg_min_df = pd.read_csv("data/avg_min.csv", encoding="latin1").values
    for playerseason in avg_min_df:
        season = playerseason[1]
        player = playerseason[0]
        avgmin = playerseason[2]
        if player in mp:
            mp[player][season]=avgmin
    return mp    

**SAVING AND LOADING MODELS

In [61]:
###JUMP HERE FOR SAVING (SAVE)
def save_models(box_score_model, model_players, pc_model, embeddings, team_datasets, \
                adjustment_datasets, box_score_datasets, team_abilities, team_win_prob_matrix):
    torch.save(box_score_model.state_dict(), "models/box_score_model.pt")
    torch.save(model_players.state_dict(), "models/player_adjustment_model.pt")
    torch.save(pc_model.state_dict(), "models/team_model.pt")

    with open("data/team_datasets.p", "wb") as f:
        pickle.dump(team_datasets, f)
    
    with open("data/adjustment_datasets.p", "wb") as f:
        pickle.dump(adjustment_datasets, f)
        
    with open("data/box_score_datasets.p", "wb") as f:
        pickle.dump(box_score_datasets, f)
    
    with open("data/embeddings.p","wb") as f:
        pickle.dump(embeddings, f)
    
    with open("data/team_abilities.p", "wb") as f:
        pickle.dump(team_abilities, f)
    
    with open("data/team_win_prob_matrix.p", "wb") as f:
        pickle.dump(team_win_prob_matrix, f)

In [62]:
def load_models(box_score_model=True, model_players=True, pc_model=True):
    
    box_score_model = None
    model_players = None
    pc_model = None
    
    if box_score_model:
        box_score_model = BoxScoreToEmbeddingModel(20)
        box_score_model.load_state_dict(torch.load("models/box_score_model.pt"))
        box_score_model.double()

    if model_players:
        model_players = EmbeddingAdjustmentModel()
        model_players = model_players.double()
        model_players.load_state_dict(torch.load("/Users/williamarnesen/Desktop/NBA_Model/model_y2y.pt"))
    
    if pc_model:
        pc_model = PlayerCombinerModel()
        pc_model.load_state_dict(torch.load("models/model_matchups.pt"))
        pc_model.double()
    
    return box_score_model, model_players, pc_model

In [63]:
def load_datasets(embeddings=True, box_score_datasets=True, team_datasets=True, adjustment_datasets=True, \
                 team_abilities=True, team_win_prob_matrix=True):
    
    embeddings = None,
    box_score_datasets = None
    team_datasets = None
    adjustment_datasets = None
    team_abilities = None
    team_win_prob_matrix = None
    
    if team_datasets:
        with open("data/team_datasets.p", "rb") as f:
            team_datasets = pickle.load(f)
    
    if adjustment_datasets:
        with open("data/adjustment_datasets.p", "rb") as f:
            adjustment_datasets = pickle.load(f)
    
    if box_score_datasets:
        with open("data/box_score_datasets.p", "rb") as f:
            box_score_datasets = pickle.load(f)
    
    if embeddings:
        with open("data/embeddings.p","rb") as f:
            embeddings = pickle.load(f)
    
    if team_abilities:
        with open("data/team_abilities.p", "rb") as f:
            team_abilities = pickle.load(f)
    
    if team_win_prob_matrix:
        with open("data/team_win_prob_matrix.p", "rb") as f:
            team_win_prob_matrix = pickle.load(f)
    
    return embeddings, box_score_datasets, team_datasets, adjustment_datsets, team_abilities, team_win_prob_matrix

In [272]:
save_models(box_score_model, model_players, pc_model, embeddings, team_datasets, \
                adjustment_datasets, box_score_datasets, team_abilities, team_win_prob_matrix)

In [ ]:
for player in draft:
    year = draft[player][0]
    if year == 2020 and len(player) > 0:
        rookie_team = create_generic_team_with_player(player, 48, embeddings[player]['2020'])
        proj=pc_model([rookie_team],[base_list])
        print(player,round(proj[0,0].item(),3))

In [90]:
df_p = pd.DataFrame(list(playoffs.items()),columns = ['team','playoffs']) 
df_f = pd.DataFrame(list(finals.items()),columns = ['team','finals']) 
df_c = pd.DataFrame(list(champions.items()),columns = ['team','champions']) 

df_p.to_csv("results/playoffsprob_jan9.csv")
df_f.to_csv("results/finalsprob_jan9.csv")
df_c.to_csv("results/championprob_jan9.csv")

In [715]:
###TRYING OUT ALTERNATIVE MODEL###
team_skills = {}
for team in team_to_player_map:
    team_list = []
    for player in team_to_player_map[team]:
        player_name = get_nearest_player(get_player_name(player), embeddings)
        if mp[player_name]['projected'] > 0:
            minutes_tensor = torch.tensor(mp[player_name]['projected'], dtype=torch.float32)
            predicted_plusminus = box_score_model.predict(embeddings[player_name]['current'].double())*minutes_tensor
            team_list.append(predicted_plusminus.float().detach())
        team_tensor = torch.stack(team_list)
        team_skills[team] = round(torch.sum(team_tensor).item(),2)

sorted_list = sorted([(team, team_skills[team]) for team in team_skills], key=lambda x: team_skills[x[0]], reverse=True)
sorted_list

[('BRK', 55.14),
 ('IND', 41.9),
 ('PHO', 41.31),
 ('TOR', 40.96),
 ('UTA', 40.74),
 ('LAC', 40.7),
 ('MIL', 40.05),
 ('DAL', 38.69),
 ('ATL', 38.22),
 ('POR', 37.35),
 ('BOS', 36.84),
 ('DEN', 36.32),
 ('MIA', 36.09),
 ('LAL', 35.02),
 ('PHI', 34.11),
 ('CHO', 32.6),
 ('SAS', 30.34),
 ('HOU', 29.83),
 ('SAC', 29.78),
 ('GSW', 28.93),
 ('CLE', 27.21),
 ('CHI', 25.95),
 ('NOP', 25.11),
 ('MEM', 24.35),
 ('ORL', 22.28),
 ('WAS', 22.14),
 ('MIN', 21.98),
 ('OKC', 18.66),
 ('NYK', 12.32),
 ('DET', 2.13)]

In [661]:
def get_positions():
    bypos = pd.read_csv("data/minutes_by_position.csv", encoding="latin1").values
    positions = {}
    for i in range(len(bypos)):
        player = bypos[i,0]
        pg = bypos[i,2]
        sg = bypos[i,3]
        sf = bypos[i,4]
        pf = bypos[i,5]
        c = bypos[i,6]
        positions[player]={'pg': pg, 'sg': sg, 'sf': sf, 'pf': pf, 'c': c}
    return positions

def get_plain(name):
    if re.search('Nikola Jok',name):
        name = "Nikola Jokic"
    if re.search('Nikola V',name):
        name = "Nikola Vucevic"
    if re.search('Kristaps',name):
        name = "Kristaps Porzingis"
    if re.search('Luka D', name):
        name = "Luka Doncic"
    if re.search('Goran',name):
        name = "Goran Dragic"
    return name

def get_awards(year=2021):
    indices = [IDX_2010, IDX_2011, IDX_2012, IDX_2013, IDX_2014, IDX_2015, IDX_2016, IDX_2017, IDX_2018, IDX_2019, IDX_2020, IDX_2021]
    relevant_index = indices[year-2010]
    war, ind_game_war = season_leaders(relevant_index, keep_print = False)
    sorted_list_mvp = sorted([(player, war[player]) for player in war], key=lambda x: war[x[0]], reverse=True)
    mvp = get_plain(sorted_list_mvp[0][0])
    
    roty, roty_team = rookie_of_the_year(sorted_list_mvp,keep_print=False,year=year-1)
    
    if year == 2021:
        field_curr = 'current'
        field_mp = 'projected'
    else:
        field_curr = str(year)
        field_mp = year
        
    field_base = str(year-1)
    sorted_list_mip = most_improved_player(keep_print=False, field_curr = field_curr, field_base = field_base, field_mp = field_mp)
    #most_improved_player(keep_print=True, field_curr = 'current', field_base = '2020', field_mp = 'projected')
    
    
    mip = sorted_list_mip[0][0]
    mip_check = 0
    mip_idx = 0
    
    while mip_check ==0:
        if mip in draft:
            if draft[mip][0]>=2019:
                mip_idx+=1
                mip = sorted_list_mip[mip_idx][0]
            else:
                mip_check = 1
        else:
            mip_check = 1
    first_team, second_team, third_team = get_all_nba(sorted_list_mvp)
    
    if year == 2021:
        east_allstars, west_allstars = get_all_stars()
    
    sorted_games = get_best_game(keep_print=False, ind_game_war = ind_game_war)
    best_game = sorted_games[0][0]
    
    print("MVP:", mvp)
    print("ROTY:", roty)
    print("MIP:", mip)
    print('\n')
    print("All-NBA First Team:", first_team)
    print("All-NBA Second Team:", second_team)
    print("All-NBA Third Team:", third_team)
    print('\n')
    if year == 2021:
        print("East All-Stars:", east_allstars)
        print('\n')
        print("West All-Stars:", west_allstars)
        print('\n')
    print('All-Rookie First-Team', roty_team)
    print('\n')
    print('Best Game:', best_game)
    

In [774]:
get_awards(year=2021)



MVP: Nikola Jokic
ROTY: LaMelo Ball
MIP: Chris Boucher


All-NBA First Team: ['Nikola Jokic', 'Julius Randle', 'Paul George', 'Luka Doncic', 'LeBron James']
All-NBA Second Team: ['Nikola Vucevic', 'Khris Middleton', 'Kevin Durant', 'Kawhi Leonard', 'Damian Lillard']
All-NBA Third Team: ['Domantas Sabonis', 'Stephen Curry', 'CJ McCollum', 'James Harden', 'Fred VanVleet']


East All-Stars: ['Khris Middleton', 'Nikola Vucevic', 'Julius Randle', 'Kevin Durant', 'Jrue Holiday', 'Domantas Sabonis', 'Larry Nance', 'James Harden', 'Giannis Antetokounmpo', 'Malcolm Brogdon', 'Terry Rozier', 'Zach LaVine']


West All-Stars: ['Nikola Jokic', 'LeBron James', 'Paul George', 'CJ McCollum', 'Kawhi Leonard', 'Harrison Barnes', 'Damian Lillard', 'Luka Doncic', 'Anthony Davis', 'Christian Wood', 'Stephen Curry', 'Dejounte Murray']


All-Rookie First-Team ['LaMelo Ball', 'Tyrese Haliburton', 'Deni Avdija', 'Devin Vassell', 'Xavier Tillman']


Best Game: Kevin Durant 202101200CLE


In [775]:
get_awards(year=2020)

MVP: LeBron James
ROTY: Ja Morant
MIP: Devonte' Graham


All-NBA First Team: ['LeBron James', 'Damian Lillard', 'James Harden', 'Rudy Gobert', 'Giannis Antetokounmpo']
All-NBA Second Team: ['Nikola Jokic', 'Chris Paul', 'Luka Doncic', 'Bam Adebayo', 'Anthony Davis']
All-NBA Third Team: ['Kyle Lowry', 'Nikola Vucevic', 'Ben Simmons', 'Domantas Sabonis', 'Trae Young']


All-Rookie First-Team ['Ja Morant', 'Brandon Clarke', 'Tyler Herro', 'Rui Hachimura', 'Cameron Johnson']


Best Game: James Harden 202007310DAL


In [776]:
get_awards(year=2019)

MVP: James Harden
ROTY: Deandre Ayton
MIP: Luka DonÄiÄ


All-NBA First Team: ['James Harden', 'Paul George', 'Rudy Gobert', 'Giannis Antetokounmpo', 'Kevin Durant']
All-NBA Second Team: ['Nikola Jokic', 'Stephen Curry', 'Russell Westbrook', 'Ben Simmons', 'LeBron James']
All-NBA Third Team: ['Damian Lillard', 'Nikola Vucevic', 'Bradley Beal', 'Tobias Harris', 'Anthony Davis']


All-Rookie First-Team ['Deandre Ayton', 'Trae Young', 'Mikal Bridges', 'Mitchell Robinson', 'Shai Gilgeous-Alexander']


Best Game: Bradley Beal 201901130WAS


In [777]:
get_awards(year=2018)

MVP: LeBron James
ROTY: Lonzo Ball
MIP: Ben Simmons


All-NBA First Team: ['LeBron James', 'Karl-Anthony Towns', 'James Harden', 'Anthony Davis', 'Kevin Durant']
All-NBA Second Team: ['Nikola Jokic', 'Kyle Lowry', 'Damian Lillard', 'Giannis Antetokounmpo', 'Draymond Green']
All-NBA Third Team: ['Russell Westbrook', 'Ben Simmons', 'Andre Drummond', 'Chris Paul', 'Joe Ingles']


All-Rookie First-Team ['Lonzo Ball', 'Lauri Markkanen', 'Jayson Tatum', 'Donovan Mitchell', 'Kyle Kuzma']


Best Game: LeBron James 201802070CLE


In [778]:
get_awards(year=2017)

MVP: James Harden
ROTY: Malcolm Brogdon
MIP: Dennis SchrÃ¶der


All-NBA First Team: ['James Harden', 'LeBron James', 'Stephen Curry', 'Kevin Durant', 'Giannis Antetokounmpo']
All-NBA Second Team: ['Russell Westbrook', 'Draymond Green', 'Karl-Anthony Towns', 'Chris Paul', 'Jimmy Butler']
All-NBA Third Team: ['Rudy Gobert', 'Kyle Lowry', 'John Wall', 'Anthony Davis', 'Kawhi Leonard']


All-Rookie First-Team ['Malcolm Brogdon', 'Tyler Ulis', 'Caris LeVert', 'Cheick Diallo', 'Brice Johnson']


Best Game: James Harden 201612310HOU


In [779]:
get_awards(year=2016)

MVP: Stephen Curry
ROTY: Karl-Anthony Towns
MIP: Gary Harris


All-NBA First Team: ['Stephen Curry', 'Draymond Green', 'Kevin Durant', 'Chris Paul', 'Russell Westbrook']
All-NBA Second Team: ['James Harden', 'LeBron James', 'DeAndre Jordan', 'Kyle Lowry', 'Kawhi Leonard']
All-NBA Third Team: ['Rajon Rondo', 'John Wall', 'Paul Millsap', 'Ricky Rubio', 'Kemba Walker']


All-Rookie First-Team ['Karl-Anthony Towns', 'Willie Cauley-Stein', 'Rondae Hollis-Jefferson', 'Josh Richardson', 'Rakeem Christmas']


Best Game: Draymond Green 201601020GSW


In [780]:
get_awards(year=2015)


MVP: Chris Paul
ROTY: Elfrid Payton
MIP: Rudy Gobert


All-NBA First Team: ['Chris Paul', 'Stephen Curry', 'DeAndre Jordan', 'Anthony Davis', 'Draymond Green']
All-NBA Second Team: ['James Harden', 'John Wall', 'Damian Lillard', 'Ty Lawson', 'LeBron James']
All-NBA Third Team: ['Kevin Love', 'Pau Gasol', 'Jimmy Butler', 'Tyson Chandler', 'Kyle Korver']


All-Rookie First-Team ['Elfrid Payton', 'Marcus Smart', 'Jordan Clarkson', 'Julius Randle', 'Bruno Caboclo']


Best Game: Damian Lillard 201412230OKC


In [781]:
get_awards(year=2014)

MVP: LeBron James
ROTY: Michael Carter-Williams
MIP: DeAndre Jordan


All-NBA First Team: ['LeBron James', 'Kevin Durant', 'Kevin Love', 'Stephen Curry', 'DeAndre Jordan']
All-NBA Second Team: ['Chris Paul', 'Kyle Lowry', 'Nicolas Batum', 'Joakim Noah', 'Dirk Nowitzki']
All-NBA Third Team: ['James Harden', 'Paul George', 'John Wall', 'Ricky Rubio', 'Andre Drummond']


All-Rookie First-Team ['Michael Carter-Williams', 'Trey Burke', 'Ryan Kelly', 'Victor Oladipo', 'Gorgui Dieng']


Best Game: Kevin Durant 201312010OKC


In [782]:
get_awards(year=2013)

MVP: LeBron James
ROTY: Damian Lillard
MIP: Nikola VuÄeviÄ


All-NBA First Team: ['LeBron James', 'Kevin Durant', 'Chris Paul', 'Stephen Curry', 'Deron Williams']
All-NBA Second Team: ['Nicolas Batum', 'James Harden', 'Al Horford', 'Paul George', 'Mike Conley']
All-NBA Third Team: ['Marc Gasol', 'David Lee', 'Kobe Bryant', 'Chandler Parsons', 'Paul Pierce']


All-Rookie First-Team ['Damian Lillard', 'Anthony Davis', 'Andre Drummond', 'Bradley Beal', 'Darius Johnson-Odom']


Best Game: Stephen Curry 201302270NYK


In [783]:
get_awards(year=2012)

MVP: Chris Paul
ROTY: Isaiah Thomas
MIP: Jeff Teague


All-NBA First Team: ['Chris Paul', 'LeBron James', 'Kevin Durant', 'Andre Iguodala', 'Kevin Love']
All-NBA Second Team: ['Steve Nash', 'Pau Gasol', 'Dwight Howard', 'Rajon Rondo', 'Tyson Chandler']
All-NBA Third Team: ['Marc Gasol', 'JosÌ© CalderÌ_n', 'Mike Conley', 'Blake Griffin', 'Josh Smith']


All-Rookie First-Team ['Isaiah Thomas', 'Kyrie Irving', 'Kawhi Leonard', 'Chandler Parsons', 'Kenneth Faried']


Best Game: Kevin Durant 201203230OKC


In [784]:
get_awards(year=2011)

MVP: Chris Paul
ROTY: Landry Fields
MIP: Kyle Lowry


All-NBA First Team: ['Chris Paul', 'LeBron James', 'Kevin Love', 'Dwight Howard', 'Pau Gasol']
All-NBA Second Team: ['Steve Nash', 'Jason Kidd', 'Deron Williams', 'Rajon Rondo', 'Al Horford']
All-NBA Third Team: ['Andre Iguodala', 'Kevin Durant', 'Lamar Odom', 'Paul Pierce', 'Raymond Felton']


All-Rookie First-Team ['Landry Fields', 'John Wall', 'Greg Monroe', 'Ed Davis', 'Hassan Whiteside']


Best Game: Monta Ellis 201103110GSW


In [792]:
war, _ = season_leaders(IDX_2021)

1 	 Nikola Jokic 	 5.689
2 	 Nikola Vucevic 	 4.687
3 	 Julius Randle 	 4.664
4 	 Paul George 	 4.542
5 	 Luka Doncic 	 4.488
6 	 LeBron James 	 4.456
7 	 Khris Middleton 	 4.434
8 	 Kevin Durant 	 4.414
9 	 Kawhi Leonard 	 4.177
10 	 Damian Lillard 	 4.095
11 	 Domantas Sabonis 	 4.068
12 	 Stephen Curry 	 4.068
13 	 CJ McCollum 	 3.965
14 	 James Harden 	 3.908
15 	 Fred VanVleet 	 3.858
16 	 Anthony Davis 	 3.761
17 	 Kyle Lowry 	 3.746
18 	 Malcolm Brogdon 	 3.734
19 	 Jrue Holiday 	 3.545
20 	 Giannis Antetokounmpo 	 3.487
21 	 Ben Simmons 	 3.414
22 	 Rudy Gobert 	 3.379
23 	 Dejounte Murray 	 3.344
24 	 Larry Nance 	 3.267
25 	 Chris Paul 	 3.224
26 	 Joel Embiid 	 3.17
27 	 DeMar DeRozan 	 3.161
28 	 Shai Gilgeous-Alexander 	 3.139
29 	 Jaylen Brown 	 3.134
30 	 Mikal Bridges 	 3.074
31 	 Harrison Barnes 	 3.059
32 	 Zach LaVine 	 3.037
33 	 Mike Conley 	 2.971
34 	 OG Anunoby 	 2.896
35 	 Jerami Grant 	 2.871
36 	 Bam Adebayo 	 2.847
37 	 Gordon Hayward 	 2.834
38 	 Christian 

In [868]:
def write_full_info():
    full_info = {}
    for playerskill in best_current_players:
        player = playerskill[0]
        player_name = get_nearest_player(player,war)
        if player_name in war:
            full_info[player_name]={'war': war[player_name]}
            full_info[player_name]['skill'] = playerskill[1]
            if player_name in player_to_team_map:
                full_info[player_name]['team'] = player_to_team_map[get_nearest_player(player_name,player_to_team_map)][0]
            else:
                player_name_aff = get_nearest_player(player_name,affiliations)
                for game in affiliations[player_name_aff]:
                    team = affiliations[player_name_aff][game]
                team_name = team_to_abbr_mapping[team]
                full_info[player_name]['team'] = team_name
    infodf = pd.DataFrame.from_dict(full_info,orient='index')
    infodf.to_csv('full_war.csv')

    return full_info

            
    

In [869]:
write_full_info()

{'LeBron James': {'war': 4.4559999999999995, 'skill': 0.384, 'team': 'LAL'},
 'Rajon Rondo': {'war': -0.11000000000000001, 'skill': 0.05, 'team': 'ATL'},
 'Dennis SchrÃ¶der': {'war': 0.17700000000000002,
  'skill': 0.043,
  'team': 'LAL'},
 'Nikola JokiÄ\x87': {'war': 5.689, 'skill': 0.374, 'team': 'DEN'},
 'JaVale McGee': {'war': -0.039999999999999994,
  'skill': -0.054,
  'team': 'CLE'},
 'Karim ManÃ©': {'war': -0.426, 'skill': -0.133, 'team': 'ORL'},
 'Eric Gordon': {'war': 0.73, 'skill': 0.028, 'team': 'HOU'},
 'Goran DragiÄ\x87': {'war': 0.4110000000000001,
  'skill': -0.187,
  'team': 'MIA'},
 'DeAndre Jordan': {'war': 0.9660000000000001, 'skill': 0.15, 'team': 'BRK'},
 'LaMarcus Aldridge': {'war': 0.279, 'skill': 0.122, 'team': 'SAS'},
 'Kyle Lowry': {'war': 3.7460000000000004, 'skill': 0.335, 'team': 'TOR'},
 'Al Horford': {'war': 0.93, 'skill': 0.198, 'team': 'OKC'},
 'Jeff Teague': {'war': -0.5509999999999999, 'skill': -0.024, 'team': 'BOS'},
 'Dario Å\xa0ariÄ\x87': {'war': -

'CLE'

dict